In [1]:
# Scraping no Site QuintoAndar
# Camila Junqueira
# Gabriel Nasser
# Rafaela Alexandre
import requests
from bs4 import BeautifulSoup
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from pprint import pprint
from geopy.geocoders import Nominatim
import geopandas as gpd
from shapely.geometry import Point

In [2]:
def getDadosCEP(cep):
		url = (f'http://www.viacep.com.br/ws/{cep}/json')
		
		req = requests.get(url)
		if req.status_code == 200:
			dados_json = json.loads(req.text)
			return dados_json
		else:
			print('Erro ao buscar CEP')

In [9]:
meu_cep = "04078012"
dados = getDadosCEP(meu_cep)
bairro = dados['bairro']
rua = dados['logradouro'].replace(' ', '-').lower()
cidade = dados['localidade'].replace(' ', '-').lower()
rua + "-" + cidade

'avenida-divino-salvador-são-paulo'

In [34]:
bairro = bairro.replace(" ", "-").lower()
bairro

'planalto-paulista'

In [35]:
qa = "https://www.quintoandar.com.br/comprar/imovel/{bairro}-sao-paulo-sp-brasil?pagina=10"

In [44]:
def scraper(bairro):
    driver = webdriver.Chrome()

    driver.get('https://www.quintoandar.com.br/comprar/imovel/'+bairro+'-sao-paulo-sp-brasil')
    time.sleep(5)

    for i in range(1):
        try:
            ver_mais_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Ver mais')]")
            ver_mais_button.click()  # Clica no botão
            time.sleep(3)  # Aguarde o conteúdo carregar após o clique
        except:
            pass

    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    soup = soup.find('html')
    return soup

In [45]:
moema = scraper(bairro)
pprint(moema)

<html lang="pt-BR"><head><style>.LGLeeN-keyboard-shortcuts-view{display:-webkit-box;display:-webkit-flex;display:-moz-box;display:-ms-flexbox;display:flex}.LGLeeN-keyboard-shortcuts-view table,.LGLeeN-keyboard-shortcuts-view tbody,.LGLeeN-keyboard-shortcuts-view td,.LGLeeN-keyboard-shortcuts-view tr{background:inherit;border:none;margin:0;padding:0}.LGLeeN-keyboard-shortcuts-view table{display:table}.LGLeeN-keyboard-shortcuts-view tr{display:table-row}.LGLeeN-keyboard-shortcuts-view td{-moz-box-sizing:border-box;box-sizing:border-box;display:table-cell;color:#000;padding:6px;vertical-align:middle;white-space:nowrap}.LGLeeN-keyboard-shortcuts-view td:first-child{text-align:end}.LGLeeN-keyboard-shortcuts-view td kbd{background-color:#e8eaed;border-radius:2px;border:none;-moz-box-sizing:border-box;box-sizing:border-box;color:inherit;display:inline-block;font-family:Google Sans Text,Roboto,Arial,sans-serif;line-height:16px;margin:0 2px;min-height:20px;min-width:20px;padding:2px 4px;positio

In [46]:
soup = BeautifulSoup(moema, 'html.parser')
casas = soup.find_all(attrs={"data-testid": "house-card-container"})
casas[0]

TypeError: 'NoneType' object is not callable

In [47]:
casas[0]

<div data-testid="house-card-container">
<a class="StyledLink_styledLink__P_6FN" href="/imovel/894366693/comprar/casa-4-quartos-planalto-paulista-sao-paulo?from_route=%22search_results%22&amp;house_tags=newAd&amp;search_id=%22115b0011-dfc2-45d7-99e2-af96d899c457%22&amp;search_rank=%7B%22sortMode%22%3A%22relevance%22%2C%22searchMode%22%3A%22list%22%2C%22resultsOrigin%22%3A%22search%22%2C%22rank%22%3A0%2C%22personalization%22%3Atrue%7D" target="_blank" title="Casa à venda com 4 quartos, 150m² em Planalto Paulista, São Paulo">
<div aria-label="Casa . Anúncio novo. Planalto Paulista, São Paulo, Rua Mocoembu. 150 metros quadrados, 4 quartos, 2 vagas de garagem.  R$ 334 Condo. + IPTU, R$ 837.900 ." class="Cozy__FindHouseCard-Container qHIgfb CsxKHh" role="group" tabindex="0">
<div aria-live="polite" class="Cozy__CardBlock-Container _2pfuCF znH6Fs">
<div class="Cozy__CardRow-Container oVdjIf" role="complementary" style="--card-row-top: 0; --card-row-bottom: 0; --card-row-left: 0; --card-row-r

In [58]:
bairro = casas[0].find("h2", class_="CozyTypography xih2fc _72Hu5c Ci-jp3").text
bairro = bairro.replace('\n','')
bairro = bairro.replace(" · ", ", ")
bairro = bairro.lstrip()
bairro = bairro.rstrip()
bairro

'Rua Mocoembu, Planalto Paulista, São Paulo'

In [60]:
def find_endereco(imovel):
    bairro = imovel.find("h2", class_="CozyTypography xih2fc _72Hu5c Ci-jp3").text
    bairro = bairro.replace('\n','')
    bairro = bairro.replace(" · ", ", ")
    bairro = bairro.lstrip()
    endereco = bairro.rstrip()
    return endereco

In [67]:
def find_bairro(imovel):
    bairro = imovel.find("h2", class_="CozyTypography xih2fc _72Hu5c Ci-jp3").text
    bairro = bairro.replace('\n','')
    bairro = bairro.replace(" · ", ", ")
    bairro = bairro.lstrip()
    bairro = bairro.rstrip()
    bairro = bairro.split(", ")[1]
    return bairro

In [68]:
find_bairro(casas[0])

'Planalto Paulista'

In [9]:
rua = casas[0].find("div", class_="Cozy__FindHouseCard-Container").text
rua = rua.split("São Paulo")[0].split('\n\n')[-1].lstrip()
rua = rua.split(",")[0]
rua

'Avenida Ireré'

In [10]:
def find_rua_imovel(imovel):
    rua = imovel.find("div", class_="Cozy__FindHouseCard-Container").text
    rua = rua.split("São Paulo")[0].split('\n\n')[-1].lstrip()
    rua = rua.split(",")[0]
    return rua

In [11]:
valor = casas[0].find("div", class_="Cozy__FindHouseCard-Container").text
valor = valor.split("R$")[1]
valor = re.sub(r'\D', '', valor)
valor

'450000'

In [12]:
img_tag = casas[0].find('img', class_='ProgressiveImage_image__1QoR0')
alt_text = img_tag['alt']
alt_text = alt_text.split("m²")[0].split(" ")
alt_text[-1]


'65'

In [13]:
def find_preco_imovel(imovel):
    valor = imovel.find("div", class_="Cozy__FindHouseCard-Container").text
    valor = valor.split("R$")[1]
    return re.sub(r'\D', '', valor)

In [14]:
find_preco_imovel(casas[0])

'450000'

In [15]:
def find_m2_imovel(imovel):
    img_tag = imovel.find('img', class_='ProgressiveImage_image__1QoR0')
    alt_text = img_tag['alt']
    alt_text = alt_text.split("m²")[0].split(" ")
    return alt_text[-1]
    

In [16]:
find_m2_imovel(casas[0])

'65'

In [17]:
for i in range(0, 30):
    print("Preco: " + find_preco_imovel(casas[i]))
    print("m2: " + find_m2_imovel(casas[i]))
    print("-"*10)

Preco: 450000
m2: 65
----------
Preco: 387000
m2: 50
----------
Preco: 593000
m2: 80
----------
Preco: 1490000
m2: 210
----------
Preco: 400000
m2: 70
----------
Preco: 675000
m2: 144
----------
Preco: 799000
m2: 120
----------
Preco: 765000
m2: 140
----------
Preco: 610000
m2: 95
----------
Preco: 455000
m2: 75
----------
Preco: 460000
m2: 96
----------
Preco: 445000
m2: 69
----------
Preco: 599000
m2: 128
----------
Preco: 1400000
m2: 220
----------
Preco: 800000
m2: 150
----------
Preco: 1427000
m2: 205
----------
Preco: 200000
m2: 18
----------
Preco: 800000
m2: 150
----------
Preco: 1900000
m2: 320
----------
Preco: 438000
m2: 60
----------
Preco: 320000
m2: 27
----------
Preco: 1100000
m2: 230
----------
Preco: 1425000
m2: 250
----------
Preco: 426000
m2: 58
----------
Preco: 2975000
m2: 457
----------
Preco: 1240000
m2: 130
----------
Preco: 1060000
m2: 300
----------
Preco: 355000
m2: 32
----------
Preco: 535000
m2: 120
----------
Preco: 430000
m2: 57
----------


In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

In [4]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cep TEXT NOT NULL,
    setor TEXT NOT NULL,
    rua TEXT NOT NULL,
    bairro TEXT NOT NULL,
    cidade TEXT NOT NULL,
    estado TEXT NOT NULL,
    regiao TEXT NOT NULL,
    preco FLOAT NOT NULL,
    m2 FLOAT NOT NULL,
    preco_m2 FLOAT NOT NULL
)
''')

In [21]:
def add_data(cep, setor, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2):
    cursor.execute('''
    INSERT INTO data (cep, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (cep, setor, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2))
    conn.commit()

In [22]:
def remove_data(id):
    cursor.execute('''
    DELETE FROM data WHERE id = ?
    ''', (id,))
    conn.commit()

In [23]:
def get_id(cep):
    cursor.execute('''
    SELECT id FROM data WHERE cep = ?
    ''', (cep,))
    data = cursor.fetchall()
    return data

In [ ]:
def geocode_address(address):
    geolocator = Nominatim(user_agent="Creating a conversion table for São Paulo")
    location = geolocator.geocode(address)
    
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

In [ ]:
def find_sector(coordenadas, setores_gdf):
    # Criar um ponto com a lat e long fornecida
    lat, lon = coordenadas
    point = Point(lon, lat)  # Note que o formato é (longitude, latitude)
    
    # Verificar qual setor contém o ponto
    setor_encontrado = setores_gdf[setores_gdf.contains(point)]
    
    if not setor_encontrado.empty:
        return setor_encontrado
    else:
        return None

In [24]:
def extract_data(cep):
    dados = getDadosCEP(cep)
    bairro = dados['bairro']
    cidade = dados['localidade']
    estado = dados['uf']
    regiao = dados['regiao']
    bairro = bairro.replace(" ", "-").lower()
    soup = scraper(bairro)
    soup = BeautifulSoup(soup, 'html.parser')
    casas = soup.find_all(attrs={"data-testid": "house-card-container"})
    for casa in casas:
        preco = find_preco_imovel(casa)
        m2 = find_m2_imovel(casa)
        rua = find_rua_imovel(casa)
        preco_m2 = float(preco) / float(m2)
        
        # Check if the data already exists
        cursor.execute('''
        SELECT id FROM data 
        WHERE cep = ? AND bairro = ? AND preco = ? AND m2 = ?
        ''', (cep, bairro, preco, m2))
        
        if cursor.fetchone() is None:
            # Data doesn't exist, so insert it
            add_data(cep, rua, bairro, cidade, estado, regiao, preco, m2, preco_m2)

In [25]:
extract_data("04078012")

In [26]:
def delete_all():
    cursor.execute('''
    DELETE FROM data
    ''')
    conn.commit()

In [27]:
# delete_all()
# Limpar banco de dados para teste 

In [28]:
def precos_medios():
    cursor.execute('''
    SELECT bairro, AVG(preco_m2) AS preco_medio_m2
    FROM data
    GROUP BY bairro
    ''')
    return cursor.fetchall()

In [29]:
extract_data("04119010")

In [30]:
precos_medios()

[('moema', 15195.022755120612),
 ('planalto-paulista', 7944.554860997816),
 ('vila-dom-pedro-i', 10813.997178278072),
 ('vila-mariana', 1731168.6372411228)]

In [31]:
for bairro, preco_medio_m2 in precos_medios():
    print(f"Bairro: {bairro} - Preço médio por m²: R${preco_medio_m2:.2f}")

Bairro: moema - Preço médio por m²: R$15195.02
Bairro: planalto-paulista - Preço médio por m²: R$7944.55
Bairro: vila-dom-pedro-i - Preço médio por m²: R$10814.00
Bairro: vila-mariana - Preço médio por m²: R$1731168.64


In [10]:
url = "https://www.quintoandar.com.br/comprar/imovel/planalto-paulista-sao-paulo-sp-brasil"

In [11]:
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
soup

<!DOCTYPE html>
<html lang="pt-BR"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="yes" name="apple-mobile-web-app-capable"/><meta content="QuintoAndar" name="apple-mobile-web-app-title"/><meta content="default" name="apple-mobile-web-app-status-bar-style"/><link href="https://cozy-assets.quintoandar.com.br/cozy-static/v1/latest/launcher-icons/ic_launcher_ios_120x120.png" rel="apple-touch-icon"/><link href="https://cozy-assets.quintoandar.com.br/cozy-static/v1/latest/launcher-icons/ic_launcher_ios_152x152.png" rel="apple-touch-icon" sizes="152x152"/><link href="https://cozy-assets.quintoandar.com.br/cozy-static/v1/latest/launcher-icons/ic_launcher_ios_167x167.png" rel="apple-touch-icon" sizes="167x167"/><link href="https://cozy-assets.quintoandar.com.br/cozy-static/v1/latest/launcher-icons/ic_launcher_ios_180x180.png" rel="apple-touch-icon" sizes="180x180"/><link href="https://cozy-assets.quintoandar.com.br/cozy-static/v1

In [9]:
numero = soup.find("div", class_="Toolbar_title__hZZcF").text.split(" ")[0]

In [10]:
numero

'2.056'

In [13]:
int(numero.replace('.', ''))//12

171

In [22]:
bairro = soup.find_all("h2", class_="CozyTypography xih2fc _72Hu5c Ci-jp3")

In [24]:
bairro[3].text.replace("· ", ",")

'Alameda dos Uapês, Planalto Paulista ,São Paulo'